# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [45]:

# Load data
df = pd.read_csv('Dataset_clustering.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13608 entries, 0 to 13607
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   is_paid                       13608 non-null  bool   
 1   num_subscribers               13608 non-null  float64
 2   avg_rating                    13608 non-null  float64
 3   num_reviews                   13608 non-null  float64
 4   num_published_lectures        13608 non-null  float64
 5   num_published_practice_tests  13608 non-null  float64
 6   discount_price__amount        13608 non-null  float64
 7   price_detail__amount          13608 non-null  float64
 8   cluster                       13608 non-null  int64  
dtypes: bool(1), float64(7), int64(1)
memory usage: 863.9 KB


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [46]:
# Buat salinan data biar nggak ngubah data asli
data = df.copy()

# Buat instance MinMaxScaler
scaler = MinMaxScaler()

# Pilih semua kolom numerik kecuali kolom target ('cluster')
numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns.drop('cluster')

# Normalisasi semua kolom numerik
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Pisahkan fitur (X) dan target (y)
X = data.drop(columns=['cluster'])
y = data['cluster']

# Split data menjadi training dan test set (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tampilkan bentuk set pelatihan dan set uji untuk memastikan split
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")


Training set shape: X_train=(10886, 8), y_train=(10886,)
Test set shape: X_test=(2722, 8), y_test=(2722,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [47]:
# Bagian 1: Pelatihan Model
# Definisikan dan latih setiap model secara terpisah
knn = KNeighborsClassifier().fit(X_train, y_train)
svm = SVC().fit(X_train, y_train)
nb = GaussianNB().fit(X_train, y_train)

print("Model training selesai.")


Model training selesai.


Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [48]:
# Fungsi untuk mengevaluasi model dengan confusion matrix
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    
    # Pastikan confusion matrix memiliki ukuran yang sesuai untuk pembacaan TP, TN, FP, FN
    if cm.shape == (2, 2):  # Hanya jika klasifikasi biner
        tn, fp, fn, tp = cm.ravel()
    else:
        tn, fp, fn, tp = None, None, None, None  # Untuk kasus multi-class

    results = {
        'Confusion Matrix': cm,
        'True Positive (TP)': tp,
        'False Positive (FP)': fp,
        'False Negative (FN)': fn,
        'True Negative (TN)': tn,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred, average='weighted')
    }
    
    return results

# Fungsi untuk evaluasi cross-validation
def cross_validate_model(model, X, y, cv=5):
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    return scores.mean(), scores.std()

# Simpan model dalam dictionary untuk menghindari kesalahan `eval()`
models = {
    'K-Nearest Neighbors (KNN)': knn,
    'Support Vector Machine (SVM)': svm,
    'Naive Bayes (NB)': nb
}

# Mengevaluasi setiap model
results = {name: evaluate_model(model, X_test, y_test) for name, model in models.items()}

# Buat DataFrame untuk menyimpan hasil evaluasi
summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Cross-Validation Mean', 'Cross-Validation Std'])

# Isi DataFrame dengan hasil evaluasi
rows = []
for model_name, model in models.items():
    metrics = results[model_name]
    mean_cv, std_cv = cross_validate_model(model, X, y, cv=5)

    rows.append({
        'Model': model_name,
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score'],
        'Cross-Validation Mean': mean_cv,
        'Cross-Validation Std': std_cv
    })

# Konversi daftar ke DataFrame
summary_df = pd.DataFrame(rows)

# Tampilkan hasil evaluasi
print(summary_df)


                          Model  Accuracy  Precision    Recall  F1-Score  \
0     K-Nearest Neighbors (KNN)  0.998898   0.998899  0.998898  0.998892   
1  Support Vector Machine (SVM)  0.997796   0.997801  0.997796  0.997773   
2              Naive Bayes (NB)  0.996326   0.996571  0.996326  0.996386   

   Cross-Validation Mean  Cross-Validation Std  
0               0.997869              0.004080  
1               0.996914              0.005301  
2               0.990447              0.003221  


Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [49]:
# ================================
# 1. Hyperparameter Tuning untuk KNN
# ================================
knn_params = {
    'n_neighbors': range(1, 21),  # Jumlah tetangga dari 1 hingga 20
    'weights': ['uniform', 'distance'],  # Cara memberi bobot tetangga
    'metric': ['euclidean', 'manhattan']  # Jenis metrik jarak
}

knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5, scoring='accuracy', n_jobs=-1)
knn_grid.fit(X_train, y_train)
best_knn = knn_grid.best_estimator_

print("Best KNN Params:", knn_grid.best_params_)

# ================================
# 2. Hyperparameter Tuning untuk SVM
# ================================
svm_params = {
    'C': [0.1, 1, 10, 100],  # Regularisasi
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Jenis kernel
    'gamma': ['scale', 'auto']  # Parameter gamma
}

svm_grid = GridSearchCV(SVC(), svm_params, cv=5, scoring='accuracy', n_jobs=-1)
svm_grid.fit(X_train, y_train)
best_svm = svm_grid.best_estimator_

print("Best SVM Params:", svm_grid.best_params_)

# ================================
# 3. Hyperparameter Tuning untuk Naïve Bayes (Menggunakan RandomizedSearchCV)
# ================================
nb_params = {
    'var_smoothing': uniform(1e-9, 1e-2)  # Mencoba nilai dari 1e-9 hingga 1e-2
}

nb_random = RandomizedSearchCV(GaussianNB(), nb_params, cv=5, scoring='accuracy', n_iter=20, random_state=42, n_jobs=-1)
nb_random.fit(X_train, y_train)
best_nb = nb_random.best_estimator_

print("Best NB Params:", nb_random.best_params_)


Best KNN Params: {'metric': 'euclidean', 'n_neighbors': 4, 'weights': 'distance'}
Best SVM Params: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Best NB Params: {'var_smoothing': np.float64(0.009699099521619943)}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [50]:
# Dictionary model terbaik setelah tuning
best_models = {
    'K-Nearest Neighbors (KNN)': best_knn,
    'Support Vector Machine (SVM)': best_svm,
    'Naive Bayes (NB)': best_nb
}

# Evaluasi ulang model setelah tuning
tuned_results = {
    model_name: evaluate_model(model, X_test, y_test)
    for model_name, model in best_models.items()
}

# Evaluasi dengan Cross-Validation
tuned_summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Cross-Validation Mean', 'Cross-Validation Std'])

tuned_rows = []
for model_name, model in best_models.items():
    mean_cv, std_cv = cross_validate_model(model, X, y, cv=5)

    tuned_rows.append({
        'Model': model_name,
        'Accuracy': tuned_results[model_name]['Accuracy'],
        'Precision': tuned_results[model_name]['Precision'],
        'Recall': tuned_results[model_name]['Recall'],
        'F1-Score': tuned_results[model_name]['F1-Score'],
        'Cross-Validation Mean': mean_cv,
        'Cross-Validation Std': std_cv
    })

# Konversi ke DataFrame
tuned_summary_df = pd.DataFrame(tuned_rows)

# Tampilkan hasil evaluasi setelah tuning
print(tuned_summary_df)


                          Model  Accuracy  Precision    Recall  F1-Score  \
0     K-Nearest Neighbors (KNN)  1.000000   1.000000  1.000000  1.000000   
1  Support Vector Machine (SVM)  1.000000   1.000000  1.000000  1.000000   
2              Naive Bayes (NB)  0.998163   0.998226  0.998163  0.998178   

   Cross-Validation Mean  Cross-Validation Std  
0               0.998898              0.002026  
1               0.997428              0.004600  
2               0.993313              0.003953  


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.

1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).

    | Model                          | Accuracy Sebelum | Accuracy Sesudah | Cross-Validation Mean Sebelum | Cross-Validation Mean Sesudah |
    |--------------------------------|-----------------|-----------------|--------------------------------|--------------------------------|
    | K-Nearest Neighbors (KNN)      | 0.9989          | 1.0000          | 0.9979                         | 0.9989                         |
    | Support Vector Machine (SVM)   | 0.9978          | 1.0000          | 0.9969                         | 0.9974                         |
    | Naive Bayes (NB)               | 0.9963          | 0.9982          | 0.9904                         | 0.9933                         |

    ---
2. Identifikasi kelemahan model, seperti:\
    **K-Nearest Neighbors (KNN)**
   - Sebelum tuning, performanya sudah sangat tinggi (>99.8%).
   - Setelah tuning, mencapai **100% accuracy**, tetapi bisa jadi **overfitting** terhadap dataset ini.
   - Perlu diuji dengan **data baru (unseen data)** untuk memastikan tidak hanya hafal data training.

    **Support Vector Machine (SVM)**
   - Performa sebelum tuning **sangat baik** (99.78% accuracy).
   - Setelah tuning, **100% accuracy**, yang juga menimbulkan potensi **overfitting**.
   - Bisa diuji dengan **lebih banyak data** untuk validasi.

    **Naive Bayes (NB)**
   - Model ini mengalami **peningkatan kecil** setelah tuning.
   - Accuracy sebelum tuning **99.63%**, setelah tuning **99.82%**.
   - Meskipun lebih rendah dibanding KNN dan SVM, performanya tetap stabil tanpa indikasi overfitting yang kuat.
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

    - **Cek dengan data baru:** Pastikan tidak ada overfitting dengan menguji model pada dataset lain.  
    - **Lihat Confusion Matrix per kelas:** Apakah ada kelas dengan Precision/Recall rendah?  
    - **Eksperimen dengan Ensemble Learning:** Coba kombinasi beberapa model untuk melihat apakah hasil lebih stabil.  
    - **Gunakan lebih banyak data:** Jika dataset terbatas, coba perbanyak dengan augmentasi data atau pengumpulan lebih lanjut.  

**Kesimpulan:** Fine-tuning meningkatkan performa, tetapi kita harus berhati-hati terhadap **overfitting**, terutama pada KNN dan SVM.  